## Builtin Apps

PX4 regard all modules(drivers, systemcmds, modules, e.g.) as Buintin Apps in NuttShell.

### PX4 implements this using cmake

* use **px4_nuttx_generate_builtin_commands** in **src/firmware/nuttx/CMakeLists.txt**

    ```cmake
    px4_nuttx_generate_builtin_commands(
        OUT builtin_commands.c
        MODULE_LIST
        ${module_libraries}
        ${config_extra_builtin_cmds}
        )
    ```

    this cmake function is defined in **cmake/nuttx/px4_impl_nuttx.cmake**


* it generate **builtin_commands.c** for NuttShell to invoke module's entry
* **module_libraries** is set in the root CMakeLists.txt using **config_module_list** in **cmake/configs/some_board.cmake**
* **config_extra_builtin_cmds** is also set in **cmake/configs/some_board.cmake**

### Schedule priority

user can set SCHED_PRIORITY in module's CMakeLists.txt using **px4_add_module**'s para

```cmake
function(px4_add_module)
	px4_parse_function_args(
		NAME px4_add_module
		ONE_VALUE MODULE MAIN STACK STACK_MAIN STACK_MAX PRIORITY
		MULTI_VALUE COMPILE_FLAGS LINK_FLAGS SRCS INCLUDES DEPENDS
		OPTIONS EXTERNAL
		REQUIRED MODULE
		ARGN ${ARGN})
```

---

## libuavcan

* set **config_extra_libs** in "cmake/configs/some_board.cmake"
    the **uavcan** and **uavcan_stm32_driver** lib are generated by libuavcan
* use **config_extra_libs** in "src/firmware/nuttx/CMakeLists.txt"
```cmake
target_link_libraries(firmware_nuttx
-Wl,--warn-common
-Wl,--gc-sections
-Wl,--start-group
${startup_libs}
${module_libraries}
${df_driver_libs}
${config_extra_libs}
${nuttx_bootloader_wrapers}
${link_libs}
-Wl,--end-group)
```

---

## Params build process

### src/modules/systemlib/param/CMakeLists.txt
    * generate ***paramaters.xml*** using **Tools/px_process_params.py**
    * generate ***px4_parameters.c/h*** using **/px_generate_params.py**
    
    
### px_process_params.py
    * will search all ***_params.c** files
    * using **Tools/px4params/srcparser.py** to parse files
    * valid tags
    ```python
    valid_tags = set(["group", "board", "min", "max", "unit", "decimal", "increment", "reboot_required", "value", "boolean", "bit"])```
    
---
    
## Firmware Generation

### prototype
**images/*.prototype** file define the structrue of the firmware file(*.px4)

### px4_nuttx_add_firmware
this cmake function is called in **src/firmware/nuttx/CMakeLists.txt**, passed elf, parameters.xml and airframes.xml into it
```cmake
px4_nuttx_add_firmware(OUT ${fw_file}
    BOARD ${BOARD}
    EXE ${CMAKE_CURRENT_BINARY_DIR}/firmware_nuttx
    PARAM_XML ${PX4_BINARY_DIR}/parameters.xml
    AIRFRAMES_XML ${PX4_BINARY_DIR}/airframes.xml
)
```

this function is defined in **cmake/nuttx/px4_impl_nuttx.cmake**, below is the core part, the elf is convert to bin using OBJCOPY, and then, bin, parameters.xml and airframes.xml are all compressed into px4 file using **px_mkfw.py** with prototype

```cmake
add_custom_command(OUTPUT ${OUT} ${EXE}.bin
    COMMAND ${OBJCOPY} -O binary ${EXE} ${EXE}.bin
    COMMAND ${PYTHON_EXECUTABLE} ${PX4_SOURCE_DIR}/Tools/px_mkfw.py
    --prototype ${PX4_SOURCE_DIR}/Images/${BOARD}.prototype
    --git_identity ${PX4_SOURCE_DIR}
    ${extra_args}
    --image ${EXE}.bin > ${OUT}
    DEPENDS ${EXE} ${dependencies}
)
```

### ROMFS
**px4_nuttx_add_romfs** in **src/firmware/nuttx/CMakeLists.txt** do the things, this function is defined in **cmake/nuttx/px4_impl_nuttx.cmake**.
   
romfs.bin is generated using GENROMFS and then convert to obj(romfs.o), finally link to firmware as a static library